In [ ]:
!pip install mlflow optuna imbalanced-learn

In [ ]:
import os
import mlflow

# Replace with your actual username and token
os.environ["MLFLOW_TRACKING_USERNAME"] = "sreyo2004"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "803d11a88c0d9d1160cb081b5953f4d1ec5cbc44"

mlflow.set_tracking_uri("https://dagshub.com/sreyo2004/mlflow-test.mlflow")

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/Himanshu-1703/reddit-sentiment-analysis/refs/heads/main/data/reddit.csv')
df.head()

,clean_comment,category
0,family mormon have never tried explain them t...,1
1,buddhism has very much lot compatible with chr...,1
2,seriously don say thing first all they won get...,-1
3,what you have learned yours and only yours wha...,0
4,for your own benefit you may want read living ...,1


In [ ]:
# Drop missing and duplicate values
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
df = df[~(df['clean_comment'].str.strip() == '')]

# Import necessary libraries
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('wordnet')

# Define the preprocessing function
def preprocess_comment(comment):
    # Convert to lowercase
    comment = comment.lower()

    # Remove trailing and leading whitespaces
    comment = comment.strip()

    # Remove newline characters
    comment = re.sub(r'\n', ' ', comment)

    # Remove non-alphanumeric characters, except punctuation
    comment = re.sub(r"[^A-Za-z0-9\s!?.]", '', comment)

    # Remove stopwords but retain important ones for sentiment analysis
    stop_words = set(stopwords.words('english')) - {'not', 'but', 'however', 'no', 'yet'}
    comment = ' '.join([word for word in comment.split() if word not in stop_words])

    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    comment = ' '.join([lemmatizer.lemmatize(word) for word in comment.split()])

    return comment

# Apply the preprocessing function to the `clean_comment` column
df['clean_comment'] = df['clean_comment'].apply(preprocess_comment)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
mlflow.set_experiment("Exp 5 - ML Algos with HP Tuning")

<Experiment: artifact_location='mlflow-artifacts:/57a11b57c17f4fbf87cf3fa6ca38890d', creation_time=1749279996336, experiment_id='9', last_update_time=1749279996336, lifecycle_stage='active', name='Exp 5 - ML Algos with HP Tuning', tags={}>

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.sklearn
import optuna

In [ ]:
# Step 1: (Optional) Remapping - skipped since not strictly needed for Logistic Regression

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

# Step 3: TF-IDF vectorizer setup
ngram_range = (1, 3)  # Trigram
max_features = 1000  # Set max_features to 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X = vectorizer.fit_transform(df['clean_comment'])
y = df['category']

# Step 4: Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)




# Step 6: Optuna objective function for Logistic Regression
def objective_logreg(trial):
    C = trial.suggest_float('C', 1e-4, 10.0, log=True)
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])

    # LogisticRegression model setup with balanced class weight
    model = LogisticRegression(C=C, penalty=penalty, solver='liblinear', random_state=42)
    return accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))


# Step 7: Run Optuna for Logistic Regression, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_logreg, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = LogisticRegression(C=best_params['C'], penalty=best_params['penalty'], solver='liblinear', random_state=42)

    # Log the best model with MLflow, passing the algo_name as "LogisticRegression"
    log_mlflow("LogisticRegression", best_model, X_train, X_test, y_train, y_test)

# Run the experiment for Logistic Regression
run_optuna_experiment()


[I 2025-06-16 13:44:34,550] A new study created in memory with name: no-name-534f42a8-6ffb-46fc-ad81-527df35f5d33
[I 2025-06-16 13:44:34,770] Trial 0 finished with value: 0.6449328965444362 and parameters: {'C': 0.0014880230673310199, 'penalty': 'l2'}. Best is trial 0 with value: 0.6449328965444362.
[I 2025-06-16 13:44:34,945] Trial 1 finished with value: 0.33329810842227625 and parameters: {'C': 0.0009440471762144772, 'penalty': 'l1'}. Best is trial 0 with value: 0.6449328965444362.
[I 2025-06-16 13:44:35,201] Trial 2 finished with value: 0.635316495825848 and parameters: {'C': 0.00022258781580094348, 'penalty': 'l2'}. Best is trial 0 with value: 0.6449328965444362.
[I 2025-06-16 13:44:36,215] Trial 3 finished with value: 0.7676212617563141 and parameters: {'C': 3.0472278630877043, 'penalty': 'l2'}. Best is trial 3 with value: 0.7676212617563141.
[I 2025-06-16 13:44:36,462] Trial 4 finished with value: 0.6431364260805241 and parameters: {'C': 0.0009391350015400705, 'penalty': 'l2'}. B

🏃 View run LogisticRegression_SMOTE_TFIDF_Trigrams at: https://dagshub.com/sreyo2004/mlflow-test.mlflow/#/experiments/9/runs/2d00613484404530b4695a713f40aafd
🧪 View experiment at: https://dagshub.com/sreyo2004/mlflow-test.mlflow/#/experiments/9
